## Client data를 Server DB에 저장하기

In [ ]:
# common.js (static 디렉토리에 common.js 추가)
# HTML head에서 js 코드 받아오기

<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.4.1/jquery.min.js"></script>
<script src="/static/common.js"></script>

In [ ]:
# index.html / client로부터 data input을 받는 html 작성

<body>
  <div id="postingBox">
    <select name='categories' id="categoryCode">
      <option value='1'>Category 1</option>
      <option value='2'>Category 2</option>
      <option value='3'>Category 3</option>
    </select>
    <textarea id="postingBody"></textarea>
    <button onclick="post_article()">Post</button>
  <div>
</body>

In [ ]:
# client data를 server의 api로 넘겨주는 ajax 함수 작성

function post_article() {
  let post_body = $('#postingBody').val();
  let category_code = $('#categoryCode').val();

  $.ajax({
    type: "POST",
    url: "/api/postarticle",
    data: {
      'category_code_give': category_code,
      'post_body_give': post_body
    },
    success: function (response) {
      if (response['result'] == 'success') {
        window.location.reload()
      }
    }
  })
}

In [ ]:
# api를 통해 받은 client data를 server의 DB에 저장

@app.route('/api/postarticle', methods=['POST'])
def post_article():
    category_code_receive = request.form['category_code_give']
    post_body_receive = request.form['post_body_give']
    
    article = {
            'category_code': category_code_receive,
            'post_body': post_body_receive
    }
    
    db.articles.insert_one(article)
    return jsonify({'result': 'success'})

## Server Data를 Client로 불러오기

In [ ]:
# DB의 모든 data를 찾아 json 형태로 client에 return

@app.route('/api/showarticles', methods=['GET'])
def show_articles():
    articles = list(db.articles.find({},{'_id':0}))
    return jsonify({'result':'success', 'articles': articles})

In [ ]:
# return 받은 data를 html 형태로 만들어주는 JavaScript 함수 작성

function make_article(category_code, article_body) {
    let temp_html = '
	<article>
	  <a href='#'>${category_code}</a>
	  <h3>${post_body}</h3>
	</article>'
    
    return temp_html;
}

In [ ]:
# return 받은 data를 client에 보여주는 ajax 함수 작성

function show_article() {
    $('#articleBox').empty();

    $.ajax({
        type: "GET",
        url: "/api/top",
        data: {},
        success: function (response) {
          let articles = response['articles_list']
          for (let i = 0; i < articles.length; i++) {
            let article = articles[i];

            let code = article['category_code'];
            let body = article['article_body'];
            
            let append_one = make_article(code, body);
            
            $('#articleBox').append(append_one);
        }
    });
}

In [ ]:
# html 내에 data를 보여줄 영역 설정

<body>
    <div id="postingBox">...

    <div id="articleBox"></div>
</body>

In [ ]:
# Server 로딩과 동시에 show_article()를 호출하는 script 작성

<script>
    $(document).ready(function () {
        show_article();
    });
</script>